In [1]:
import k3d
import numpy as np

import sys
sys.path.insert(0, "..")

from Geometry.MPMT import MPMT

In [2]:
# Draw the components of a single mPMT using k3d - it is placed at the origin

plot = k3d.plot()

# make a standard domed mPMT
my_mpmt = MPMT('my_mpmt', kind='ME')

# pmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
color_y = 0x00ff00
origins = []
z_vecs = []
x_vecs = []
y_vecs = []
vec_length = 30. # length of x,y,z vectors to show for each pmt

# pmt faces:
color_pmt = 0xdc7633
n_point_pmt = 20
indices_pmt = []
for i in range(n_point_pmt):
    indices_pmt.append([i,(i+1)%n_point_pmt])

# led coordinates
led_origins = []
led_z_vecs = []
led_vec_length = 30. # length of z vectors to show for each led

color_led = 0x00ff00

# mpmt
color_mpmt = 0xabb2b9
baseplate_points = np.array(my_mpmt.get_xy_points('design'),dtype=np.float32)
n_point_mpmt = len(baseplate_points)
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

# draw the extent of an mpmt
plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
plot += plt_baseplate

# draw the feedthroughs
feedthrough_points = np.array(my_mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
n_point_mpmt = len(feedthrough_points)
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

plt_feedthrough = k3d.lines(feedthrough_points, indices_mpmt, indices_type='segment', color=color_mpmt)
plot += plt_feedthrough

# draw the survey C holes
for c in ['c1','c2','c3','c4']:
    survey_points = np.array(my_mpmt.get_xy_points('design', feature='survey_'+c),dtype=np.float32)
    n_point_mpmt = len(survey_points)
    indices_mpmt = []
    for i in range(n_point_mpmt):
        indices_mpmt.append([i,(i+1)%n_point_mpmt])
    plt_survey = k3d.lines(survey_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_survey
    plt_text = k3d.text(c, position=survey_points[0], reference_point='cc', size=1., label_box=False, color=color_mpmt)
    plot += plt_text

# draw the fiducial points
fiducial_points = np.array(my_mpmt.get_fiducials('design'),dtype=np.float32)
# text offset in mPMT z direction
mpmt_z = my_mpmt.get_transformed_points([0.,0.,1.],'design')
for i_f, fiducial_point in enumerate(fiducial_points):
    text_location = [fiducial_point[j] - 10.*mpmt_z[j] for j in range(3)]
    plt_text = k3d.text('f'+str(i_f+1), position=text_location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    plot += plt_text

fiducial_locations = np.array(fiducial_points, dtype=np.float32)
plt_fps = k3d.points(positions=fiducial_locations,
                    point_size=8.,
                    shader='3d',
                    color=color_mpmt)
plot += plt_fps

for i_pmt, pmt in enumerate(my_mpmt.pmts):
    p = pmt.get_placement('design')
    location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
    direction_y = np.cross(direction_z,direction_x)

    # lists to show PMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    y_vec = list(np.array(direction_y)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)
    y_vecs.append(y_vec)

    # k3d expects ndarray of float32 for segments
    circle_points = np.array(pmt.get_circle_points(n_point_pmt,'design'),dtype=np.float32)
    plt_circle = k3d.lines(circle_points, indices_pmt, indices_type='segment', color=color_pmt)
    plot += plt_circle
    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_pmt), position=location, reference_point='cc', size=1., label_box=False, color=color_pmt)
    plot += plt_text

# plot all pmt coordinate axes
plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z, head_size=50.)
plot += plt_z_vecs
plt_x_vecs = k3d.vectors(origins=origins, vectors=x_vecs, color=color_x, head_size=50.)
plot += plt_x_vecs
plt_y_vecs = k3d.vectors(origins=origins, vectors=y_vecs, color=color_y, head_size=50.)
plot += plt_y_vecs

# show the LED diffuser locations

for i_led, led in enumerate(my_mpmt.leds):
    p = led.get_placement('design')
    location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
    z_vec = list(np.array(direction_z)*led_vec_length)
    led_origins.append(location)
    led_z_vecs.append(z_vec)

    # k3d complains about the following not being float32!
    text_location = [location[j] + 10.*direction_x[j] for j in range(3)]
    plt_text = k3d.text(str(i_led), position=text_location, reference_point='cc', size=1., label_box=False, color=color_led)
    plot += plt_text

#plot led locations and axis
led_locations = np.array(led_origins, dtype=np.float32)
plt_leds = k3d.points(positions=led_locations,
                        point_size=8.,
                        shader='3d',
                        color=color_led)
plot += plt_leds

plt_led_z_vecs = k3d.vectors(origins=led_origins, vectors=led_z_vecs, color=color_z, head_size=50.)
plot += plt_led_z_vecs

plot.display()

C:\Users\Karlen\AppData\Local\Programs\Python\Python39\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()